In [ ]:
# %load ../../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
sam = holo.sam.Semi_Analytic_Model(shape=30)
hard = holo.hardening.Hard_GW()

fobs = utils.nyquist_freqs(10.0*YR, 0.2*YR)
fobs_edges = utils.nyquist_freqs_edges(10.0*YR, 0.2*YR)


In [ ]:
# Sample the SAM population using 'outlier sampling'
# use orbital frequency
fobs_orb_edges = fobs_edges / 2.0
vals, weights, edges, dens, mass = holo.sam.sample_sam_with_hardening(sam, hard, fobs_orb=fobs_orb_edges, sample_threshold=1e2, poisson_inside=True, poisson_outside=True)

In [ ]:
# use GW frequency
gff, gwf, gwb = holo.gravwaves._gws_from_samples(vals, weights, fobs_edges)

In [ ]:
fig, ax = plot.figax(figsize=[12, 8], xlabel='Frequency [yr$^{-1}$]', ylabel='c-Strain')
ax.scatter(gff, gwf)
plot.draw_hist_steps(ax, fobs_edges, gwb, yfilter=lambda xx: np.greater(xx, 0.0))

plt.show()

In [ ]:
hs, fo = holo.gravwaves._strains_from_samples(vals)

In [ ]:
nloud = 5
colors = plot._get_cmap('plasma')(np.linspace(0.05, 0.95, nloud))# print(colors)

fig, ax = plot.figax(figsize=[12, 8], xlabel='Frequency [yr$^{-1}$]', ylabel='c-Strain')
for ii in range(fobs_edges.size-1):
    # if ii < 10 or ii > 16:
    #     continue
    
    fextr = [fobs_edges[ii+jj] for jj in range(2)]
    fextr = np.asarray(fextr)
    cycles = 1.0 / np.diff(np.log(fextr))[0]

    idx = (fextr[0] <= fo) & (fo < fextr[1])
    hs_bin = hs[idx]
    fo_bin = fo[idx]    
    ww_bin = weights[idx]
    ww = ww_bin * cycles

    tot = np.sqrt(np.sum(ww * hs_bin**2))
    ax.plot(fextr*YR, tot * np.ones_like(fextr), 'k--')

    idx = np.argsort(hs_bin)[::-1]
    if any(ww_bin[idx[:nloud]] > 1):
        raise
    
    for jj, cc in enumerate(colors):
        if jj >= len(idx):
            break
        hi = idx[jj]
        lo = idx[jj+1:]
        gw_hi = np.sqrt(np.sum(ww[hi] * hs_bin[hi]**2))
        gw_lo = np.sqrt(np.sum(ww[lo] * hs_bin[lo]**2))

        fave = np.average(fo_bin[hi], weights=hs_bin[hi])
        ax.plot(fextr*YR, gw_lo * np.ones_like(fextr), color=cc, lw=0.5)
        ax.scatter(fave*YR, gw_hi, marker='.', color=cc, alpha=0.5)

plt.show()